# Test debris_tracking

A few examples to illustrate the use of the `debris_tracking` module.  Still under development.

These examples show debris being passively advected in a given velocity field defined by `u(x,y,t), v(x,y,t)` that are independent of time `t` in the examples shown here.

Running this requires `shapely` and `clawpack`, the latter for the `animation_tools` that are further illustrated in this [demo notebook](https://www.clawpack.org/gallery/_static/apps/notebooks/visclaw/animation_tools_demo.html).


In [ ]:
%matplotlib inline

In [ ]:
from pylab import *
from importlib import reload
import shapely
from shapely import plotting as shapely_plotting
from clawpack.visclaw import animation_tools
from IPython.display import display, HTML


In [ ]:
import debris_tracking
reload(debris_tracking) # for debugging

## Specifying debris

A debris object can be defined as a polygon using `debris = debris_tracking.DebrisObject()`. To specify a polygon with `ncorners` corners, specify `debris.L` as a list of `ncorners - 1` side lengths, starting from some corner 0 and going counterclockwise around the object. Also specify `debris.phi` as a set of `ncorners - 1` turning angles at each corner 1, 2, etc.  The initial (initial) position of the debris object is denoted by `debris.z`, a tuple `(x,y,theta)` where `(x,y)` are the coordinates of corner 0 and `theta` is the angle between the x-axis and the polygon side going from from corner 0 to corner 1 (which has length `debris.L[0]`).

### Rectangular debris object

We can define a rectangle by giving 3 lengths (with L[2] = L[0]) and 90 degree turning angles:

In [ ]:
debris = debris_tracking.DebrisObject()
debris.L = [2, 1.5, 2]  # lengths of first 3 sides
debris.phi = [pi/2, pi/2, pi/2]  # turning angle at each corner

# rectangle with edge 0 aligned with x-axis:
debris.z = [1,2,0]
xc,yc = debris.get_corners(close_poly=True)  # uses z = debris.z
plot(xc, yc, 'b')
grid(True)
gca().set_aspect(1)

# rotate so edge 0 is at 45 degrees:
debris.z = [1 ,2, pi/4]
xc,yc = debris.get_corners(close_poly=True)  # uses z = debris.z
plot(xc, yc, 'r')

### Equilateral triangle:

In [ ]:

debris = debris_tracking.DebrisObject()
debris.L = [1,1]
debris.phi = [2*pi/3, 2*pi/3]

# plot triangle with bottom edge flat:
debris.z = [1,2,0]
xc,yc = debris.get_corners(close_poly=True)  # uses z = debris.z0
plot(xc, yc, 'b')
grid(True)
gca().set_aspect(1)

# same triangle, rotated by 45 degrees:
debris.z = [1,2,0]
xc,yc = debris.get_corners(z=(1,2,pi/4), close_poly=True)
plot(xc, yc, 'r')

## Some convenience functions for plotting:

In [ ]:
def plot_quiver(extent, dx, dy, u, v):
    """
    Make a quiver plot of a velocity field defined by function u(x,y), v(x,y)
    Plots over the extent [x1,x2,y1,y2] on a grid with spacing dx,dy.
    Scale the arrow lengths based on the maximum velocities and mesh spacing.
    """
    xvel = arange(extent[0], extent[1]+dx/2, dx)
    yvel = arange(extent[2], extent[3]+dy/2, dy)
    Xvel,Yvel = meshgrid(xvel,yvel,indexing='xy')
    Uvel = u(Xvel,Yvel,0)
    Vvel = v(Xvel,Yvel,0)
    smax = sqrt(Uvel**2 + Vvel**2).max()
    #print(f'Scaling by 1.1*smax/dx = {1.1*smax/dx}')
    quiver(Xvel,Yvel,Uvel,Vvel,scale=1.1*smax/dx,scale_units='xy',color='gray',width=0.003)

## Shear flow

As a simple velocity field, consider shear flow where the x-velocity `u` increases linearly with `y` and the y-velocity `v` is identically zero.

Note that u,v are define as functions of `(x,y,t)` and can be time dependent, but the examples in this notebook all use velocity fields that are cosntant in time.

In [ ]:
# shear flow:
u = lambda x,y,t: y
v = lambda x,y,t: zeros(x.shape)
plot_quiver([-1,8,-1,3], 1, 0.5, u, v)

## Function to make an animation of debris motion


In [ ]:
def make_anim(debris_path_list, extent, figsize=(6,6), obst_list=[], domain=None, u=None, v=None):
    """
    :Inputs:
        - debris_path_list: a list of debris_tools.DebrisPath objects describing the path in time
            of each debris object.  This must first be computed and passed in.
        - extent: axis [x1,x2,y1,y2] for plots
        - figsize: figure size
        - obst_list: list of obstacles that were specified when computing the debris paths, a list
            of dictionaries (for adding the obstacles to the plots)
        - domain: the domain specified when computing the debris paths, a rectangle [x1,x2,y1,y2] bounded
            by solid walls the debris objects could not penetrate (for plotting these walls).
            domain == None is equivalent to domain == [-inf, inf, -inf, inf] (no walls on any side)
        - u,v: functions of (x,y,t) specifying the velocity field. If passed in, quiver plot will be added.
    """
    
    figs = [] # to accumulate frames of animation
    for n in range(nsteps+1):
        fig = figure(figsize=figsize)

        if (u is not None) and (v is not None):
            # quiver plot of velocity field:
            x1,x2,y1,y2 = extent
            if domain is not None:
                x1 = max(x1,domain[0])
                x2 = min(x2,domain[1])
                y1 = max(y1,domain[2])
                y2 = min(y2,domain[3])
            dx = (x2-x1)/20
            dy = (y2-y1)/20
            plot_quiver([x1,x2,y1,y2], dx, dy, u, v)

        # plot any obstacles:
        for obst in obst_list:
            shapely.plotting.plot_polygon(obst['polygon'], add_points=False,
                                          color='blue', alpha=0.5)

        if domain is not None:
            # plot any walls:
            x1,x2,y1,y2 = extent
            x1wall,x2wall,y1wall,y2wall = domain
            plot([x1wall,x1wall], [max(y1wall,y1),min(y2wall,y2)], 'g')
            plot([x2wall,x2wall], [max(y1wall,y1),min(y2wall,y2)], 'g')
            plot([max(x1wall,x1),min(x2wall,x2)], [y1wall,y1wall], 'g')
            plot([max(x1wall,x1),min(x2wall,x2)], [y2wall,y2wall], 'g')


        # plot debris paths:
        for debris_path in debris_path_list:
            debris = debris_path.debris
            t_n = debris_path.times[n]
            z_n = debris_path.z_path[n]
            xc,yc = debris.get_corners(z_n, close_poly=True)
            plot(xc, yc, 'b')
            
        #grid(True)
        axis(extent)
        title(f'Time {t_n} seconds')
        gca().set_aspect(1)
        figs.append(fig)
        close(fig)

    images = animation_tools.make_images(figs)
    anim = animation_tools.animate_images(images, figsize=figsize)
    return anim

## Examples

### Single debris object in shear flow

In [ ]:
debris = debris_tracking.DebrisObject()
debris.L = [1.5,1,1.5]
debris.phi = [pi/2, pi/2, pi/2]

# Experiment with changing the height y in z = (x,y,z)
debris.z = (0, 0.5, 0)

debris.advect = True  # Particle advected with flow

# list of debris particles and initial locations (as tuples z = (x,y,theta)):
debris_list = [debris]
z0_list = [db.z for db in debris_list]

# shear flow:
u = lambda x,y,t: y
v = lambda x,y,t: zeros(x.shape)
h = lambda x,y,t: 10.

# ---------------------------------
# obstacles:  Try it with and without

if 0:
    obst = debris_tracking.make_rectangular_obstacle(2,3,0.5,1)
    obst_list = [obst]
else:
    obst_list = []  # no obstacles

# ---------------------------------
# walls:  Try it with and without

if 0:
    ywall = 0.5
    domain = [-inf, inf, ywall,inf]
else:
    domain = None  # no physical domain walls

# ---------------------------------
# time stepping parameters:

t0 = 0.
nsteps = 10
dt = 0.5

# main computational loop -- compute all the debris paths:

debris_path_list = debris_tracking.make_debris_path_list(debris_list, z0_list,
                                                         obst_list, domain,
                                                         t0,dt,nsteps,h,u,v)


# ---------------------------------
# make plots:

if 1:
    debris_path = debris_path_list[0]  # only one debris object
    # plot several times on one static plot
    figure(figsize=(8,8))
    plot_quiver([-1,8,-1,3], 1, 0.5, u, v)
    c = 10*['k','r','b','g']
    for n in range(nsteps+1):
        t_n = debris_path.times[n]
        z_n = debris_path.z_path[n]
        xc,yc = debris.get_corners(z_n, close_poly=True)
        plot(xc, yc, color=c[n])
    gca().set_aspect(1)  # aspect ratio for plots
    axis([0,8,0,4])
    title('All time steps')


# make animation:

anim = make_anim(debris_path_list, [0,8,0,4], (6,4), obst_list, domain, u, v)
HTML(anim.to_jshtml())


## Shear flow with two debris objects

In [ ]:
# Rectangular debris:
debris = debris_tracking.DebrisObject()
debris.L = [1.5,1,1.5]
debris.phi = [pi/2, pi/2, pi/2]

debris.z = (1.5, 0.5, 0)
debris.advect = True  # Particle advected with flow
debris_list = [debris]

# Triangular debris:
debris = debris_tracking.DebrisObject()
debris.L = [1.2,1.2]
debris.phi = [2*pi/3, 2*pi/3]
debris.z = [0,1.1,0]

debris.advect = True
debris_list.append(debris)


# initial locations (as tuples z = (x,y,theta)):
z0_list = [db.z for db in debris_list]

# shear flow:
u = lambda x,y,t: y
v = lambda x,y,t: zeros(x.shape)
h = lambda x,y,t: 10.

# ---------------------------------
# obstacles:  Try it with and without

if 1:
    obst = debris_tracking.make_rectangular_obstacle(4,5,0.25,0.75)
    obst_list = [obst]
else:
    obst_list = []  # no obstacles

# ---------------------------------
# walls:  Try it with and without

if 1:
    ywall = 0.25
    domain = [-inf, inf, ywall,inf]
else:
    domain = None  # no physical domain walls

# ---------------------------------
# time stepping parameters:

t0 = 0.
nsteps = 25
dt = 0.25

# main computational loop -- compute all the debris paths:

debris_path_list = debris_tracking.make_debris_path_list(debris_list, z0_list,
                                                         obst_list, domain,
                                                         t0,dt,nsteps,h,u,v)
# --------------------------------
# plotting:

if 0:
    # plot all times on same figure:
    extent = [-2,8,0,4]
    figure(figsize=(8,6))
    plot_quiver([-1,8,-1,3], 1, 0.5, u, v)

    for obst in obst_list:
        shapely_plotting.plot_polygon(obst['polygon'], add_points=False,
                                      color='blue', alpha=0.5)
    

    c = 10*['k','r','b','g']
    for n in range(nsteps+1):
        for debris_path in debris_path_list:
            debris = debris_path.debris
            t_n = debris_path.times[n]
            z_n = debris_path.z_path[n]
            xc,yc = debris.get_corners(z_n, close_poly=True)
            plot(xc, yc, color=c[n], label='t = %.1f' % t_n)
    
    #legend()
    axis(extent)
    gca().set_aspect(1)
    grid(True)
    plot([0,8], [ywall,ywall], 'r')
    plot([xwall,xwall], [0,4], 'r')
    title('All frames')
    
# --------------------------------
# make animation:

anim = make_anim(debris_path_list, [0,8,0,4], (6,4), obst_list, domain, u, v)
HTML(anim.to_jshtml())


## Swirling flow

This velocity field is confined to the unit square.

In [ ]:
figure(figsize=(6,6))
u = lambda x,y,t: -2*sin(pi*y)*cos(pi*y)*sin(pi*x)**2
v = lambda x,y,t: 2*sin(pi*x)*cos(pi*x)*sin(pi*y)**2
plot_quiver([0,1,0,1], 0.05, 0.05, u, v)
gca().set_aspect(1)
xlim(0,1)
ylim(0,1)

In [ ]:
# Square debris:
debris = debris_tracking.DebrisObject()
debris.L = [0.3,0.3,0.3]
debris.phi = [pi/2, pi/2, pi/2]
debris.z = [0.6,0.2,0]

debris.advect = True
debris.rho = 900.
debris_list = [debris]


# Triangular debris:
debris = debris_tracking.DebrisObject()
debris.L = [0.4,0.4]
debris.phi = [2*pi/3, 2*pi/3]
debris.z = [0.5, 0.6, 0]

debris.advect = True
debris.rho = 900.
debris_list.append(debris)

z0_list = [db.z for db in debris_list]

# swirling flow:
u = lambda x,y,t: -2*sin(pi*y)*cos(pi*y)*sin(pi*x)**2
v = lambda x,y,t: 2*sin(pi*x)*cos(pi*x)*sin(pi*y)**2
h = lambda x,y,t: 10.

t0 = 0.
nsteps = 30
dt = 0.25

#obst = debris_tracking.make_rectangular_obstacle(2,3,0.5,1)
#obst_list = [obst]
obst_list = []

#domain = None  # no physical domain walls
domain = [0,1,0,1]

debris_path_list = debris_tracking.make_debris_path_list(debris_list, z0_list,
                                                         obst_list, domain,
                                                         t0,dt,nsteps,h,u,v)



# make animation:
extent = [-0.1,1.1,-0.1,1.1]
figsize = (6,6)
anim = make_anim(debris_path_list, extent, figsize, obst_list, domain, u, v)
HTML(anim.to_jshtml())